## Задание 1

In [51]:
import numpy as np
from typing import List, Tuple
from collections import defaultdict

In [52]:
class MyMatrix:
    __array_priority__ = 1

    def __init__(self, n: int, m: int, elements: List[Tuple[int, int, int]]):
        """
        Создает матрицу n*m.
        elements - массив троек (i, j, x) таких, что элемент [i, j] матрицы равен x.
        Остальные элементы матрицы нулевые.
        """
        assert n >= 0 and m >= 0
        used = defaultdict(bool)
        for i, j, x in elements:
            assert 0 <= i < n and 0 <= j < m, 'Индексы elements должны быть из [0..n-1] x [0..m-1]'
            assert not used[i * m + j], 'Индексы elements не должны повторяться'
            used[i * m + j] = True
        
        self.n = n
        self.m = m
        self.elements = elements
    
    def __matmul__(self, a: np.ndarray):
        assert a.shape == (self.m, 1), 'не совпадают размерности'

        res = np.zeros((self.n, 1))
        for i, j, x in self.elements:
            res[i, 0] += x * a[j, 0]
        return res
    
    def __rmatmul__(self, a: np.ndarray):
        assert a.shape == (1, self.n), 'не совпадают размерности'

        res = np.zeros((1, self.m))
        for i, j, x in self.elements:
            res[0, j] += a[0, i] * x
        return res


A = MyMatrix(3, 3, [(0, 0, 1),
                    (0, 1, 2),
                    (1, 2, 1),
                    (2, 0, 2)])
B = np.array([1, 0, 2])

"""
(1 2 0)   (1)   (1)
(0 0 1) * (0) = (2)
(2 0 0)   (2)   (2)
"""
print(A @ B.reshape(-1, 1))

"""
          (1 2 0)
(1 0 2) * (0 0 1) = (5 2 0)
          (2 0 0)
"""
print(B.reshape(1, -1) @ A)

[[1.]
 [2.]
 [2.]]
[[5. 2. 0.]]


In [53]:
class MyMatrix:
    __array_priority__ = 1

    def __init__(self, n: int, m: int, elements: List[Tuple[int, int, int]]):
        """
        Создает матрицу n*m.
        elements - массив троек (i, j, x) таких, что элемент [i, j] матрицы равен x.
        Остальные элементы матрицы нулевые.
        """
        assert n >= 0 and m >= 0
        used = defaultdict(bool)
        for i, j, x in elements:
            assert 0 <= i < n and 0 <= j < m, 'Индексы elements должны быть из [0..n-1] x [0..m-1]'
            assert not used[i * m + j], 'Индексы elements не должны повторяться'
            used[i * m + j] = True
        
        self.n = n
        self.m = m
        self.elements = elements
    
    def __matmul__(self, a: np.ndarray):
        assert a.shape == (self.m, 1), 'не совпадают размерности'

        res = np.zeros((self.n, 1))
        for i, j, x in self.elements:
            res[i, 0] += x * a[j, 0]
        return res
    
    def __rmatmul__(self, a: np.ndarray):
        assert a.shape == (1, self.n), 'не совпадают размерности'

        res = np.zeros((1, self.m))
        for i, j, x in self.elements:
            res[0, j] += a[0, i] * x
        return res


A = MyMatrix(3, 3, [(0, 0, 1),
                    (0, 1, 2),
                    (1, 2, 1),
                    (2, 0, 2)])
B = np.array([1, 0, 2])

"""
(1 2 0)   (1)   (1)
(0 0 1) * (0) = (2)
(2 0 0)   (2)   (2)
"""
print(A @ B.reshape(-1, 1))

"""
          (1 2 0)
(1 0 2) * (0 0 1) = (5 2 0)
          (2 0 0)
"""
print(B.reshape(1, -1) @ A)

[[1.]
 [2.]
 [2.]]
[[5. 2. 0.]]


## Задание 2

Минимизировать $||Ax - b|| \Leftrightarrow$ минимизировать $(Ax - b)^T(Ax - b) = (x^TA^T - b^T)(Ax - b) = x^TA^TAx - b^TAx - x^TA^Tb + b^Tb = x^TA^TAx - b^TAx - b^TAx + b^Tb = \frac{1}{2}(x^T(2A^TA)x) - (2A^Tb)^Tx + (b^Tb)$

Т.о. мы хотим решить $A^TAx = A^Tb$

Если $A^TA$ вырожденная, не получится решить задание (не получится найти ортогональные вектора для грамма-шмидта). Значит если $k$ -- количество ненулевых элементов $A$, а $(m, n)$ -- размер $A$, то $n \le k$, а значит асимптотика $O(n^2 + nk)$ нас тоже устраивает.

In [54]:
def my_matrix_from_np(a):
    return MyMatrix(a.shape[0],
                    a.shape[1],
                    [(i, j, aij) for i, row in enumerate(a)
                                 for j, aij in enumerate(row)
                                 if aij != 0])

def minimize(A: np.ndarray, b: np.ndarray):
    assert len(A.shape) == 2
    m, n = A.shape
    b = b.reshape((-1, 1))
    assert b.shape == (m, 1)
    
    A_  = my_matrix_from_np(A)
    A_T = my_matrix_from_np(A.T)

    x = np.zeros((n, 1))
    v = A_T @ (A_ @ x - b)
    d = v
    v_norm = np.dot(v.T, v)
    
    for i in range(n):
        Ad = A_T @ (A @ d)
        alpha = v_norm / np.dot(d.T, Ad)
        x = x - alpha * d
        v = v - alpha * Ad
        v_norm_new = np.dot(v.T, v)
        
        d = v + (v_norm_new / v_norm) * d
        v_norm = v_norm_new
    
    return x

In [55]:
"""
(x+2y-2)^2 + (y-1)^2 + (x+z)^2
минимум при (x=0, y=1, z=0)
"""

print(minimize(np.array([[1, 2, 0],
                         [0, 1, 0],
                         [1, 0, 1]]),
               np.array([2, 1, 0])))

[[-4.82947016e-15]
 [ 1.00000000e+00]
 [-8.32667268e-17]]
